In [24]:
clear all

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io 
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from scipy.fft import fft
import random
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



In [26]:
faultI10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_10hz.mat')
faultI30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_30hz.mat')
faultII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_10hz.mat')
faultII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_30hz.mat')
faultIII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_10hz.mat')
faultIII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_30hz.mat')
faultIV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_10hz.mat')
faultIV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_30hz.mat')
faultV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_10hz.mat')
faultV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_30hz.mat')





In [27]:
sigII30=faultII30['Acquisition_un_2_30hz']
sigII10=faultII10['Acquisition_un_2_10hz']
del faultII30,faultII10
sigI30=faultI30['Acquisition_un_1_30hz']
sigI10=faultI10['Acquisition_un_1_10hz']
del faultI30,faultI10
sigIII10=faultIII10['Acquisition_un_3_10hz']
sigIII30=faultIII30['Acquisition_un_3_30hz']
del faultIII10,faultIII30
sigIV10=faultIV10['Acquisition_un_4_10hz']
sigIV30=faultIV30['Acquisition_un_4_30hz']
del faultIV10,faultIV30
sigV10=faultV10['Acquisition_un_5_10hz']
sigV30=faultV30['Acquisition_un_5_30hz']
del faultV10,faultV30
# sigVI10=faultVI10['Acquisition_un_6_10hz']
# sigVI30=faultVI30['Acquisition_un_6_30hz']
# del faultVI10,faultVI30

In [28]:

def sliding_window(data, window_size, stride):
    """
    Applies a sliding window to the given data with the specified window size
    and stride length. Padding is added at the beginning and end of the data to
    ensure all samples are included in windows.
    """
    num_padding = window_size - stride
    data_padded = np.pad(data, ((num_padding, num_padding), (0, 0)), 'constant')
    window_data = []
    for i in range(0, len(data_padded) - window_size + 1, stride):
        window = data_padded[i:i+window_size, :]
        window_data.append(window)
    window_data = np.array(window_data)
    # window_data = window_data[:training_size, :, :]
    window_data = np.transpose(window_data, (0, 2, 1))
    return window_data

In [29]:
windowsize=1000
stride=700

In [30]:
sigtI10hz=sliding_window(sigI10,windowsize,stride)[:,1:13,:]
sigtI30hz=sliding_window(sigI30,windowsize,stride)[:,1:13,:]
sigtII10hz=sliding_window(sigII10,windowsize,stride)[:,1:13,:]
sigtII30hz=sliding_window(sigII30,windowsize,stride)[:,1:13,:]
del sigI10,sigI30,sigII10,sigII30
sigtIII30hz=sliding_window(sigIII30,windowsize,stride)[:,1:13,:]
sigtIII10hz=sliding_window(sigIII10,windowsize,stride)[:,1:13,:]
sigtIV30hz=sliding_window(sigIV30,windowsize,stride)[:,1:13,:]
sigtIV10hz=sliding_window(sigIV10,windowsize,stride)[:,1:13,:]
del sigIII30,sigIII10,sigIV30,sigIV10
sigtV30hz=sliding_window(sigV30,windowsize,stride)[:,1:13,:]
sigtV10hz=sliding_window(sigV10,windowsize,stride)[:,1:13,:]
# sigtVI30hz=sliding_window(sigVI30,windowsize,stride)
# sigtVI10hz=sliding_window(sigVI10,windowsize,stride)
# del sigV30,sigV10,sigVI30,sigVI10

In [31]:
datalable1=np.concatenate([sigtI10hz,sigtI30hz],axis=-1)

datalable2=np.concatenate([sigtII10hz,sigtII30hz],axis=-1)

datalable3=np.concatenate([sigtIII10hz,sigtIII30hz],axis=-1)

datalable4=np.concatenate([sigtIV10hz,sigtIV30hz],axis=-1)

datalabele5=np.concatenate([sigtV10hz,sigtV30hz],axis=-1)


In [32]:
#labels=np.concatenate((np.ones(np.shape(datalable1)[0]),np.zeros(np.shape(datalable1)[0]),np.ones(np.shape(datalable1)[0])*4,np.ones(np.shape(datalable1)[0])*2,np.ones(np.shape(datalable1)[0])*3),axis=0)
dataall=np.concatenate([datalable1,datalable2,datalable3,datalable4,datalabele5],axis=0)

In [33]:
np.shape(dataall)

(4305, 12, 2000)

In [34]:
# import numpy as np
# from scipy.signal import stft

# # generate example signal with shape (7530, 12, 1000)
# data = dataall

# # set STFT parameters
# window = 'hann'
# nperseg = 256
# noverlap = 128
# fs = 10000

# # compute STFT spectrogram for each channel
# spectrograms = []
# for i in range(data.shape[1]):
#     _, _, spec = stft(data[:, i, :], fs=fs, window=window, nperseg=nperseg, noverlap=noverlap)
#     spectrograms.append(np.abs(spec))

# # stack spectrograms along the channel axis to get shape (7530, 12, num_freq_bins, num_time_bins)
# spectrograms = np.stack(spectrograms, axis=1)





In [35]:
# np.shape(spectrograms)

In [36]:
# mean = np.mean(dataall, axis=(0, 1, 2, 3))
# std = np.std(spectrograms, axis=(0, 1, 2, 3))
# normalized_data = (spectrograms - mean) / std
data=dataall


In [37]:
np.shape(data)

(4305, 12, 2000)

In [38]:
labels=np.concatenate((np.ones(np.shape(datalable1)[0]),np.zeros(np.shape(datalable1)[0]),np.ones(np.shape(datalable1)[0])*4,np.ones(np.shape(datalable1)[0])*2,np.ones(np.shape(datalable1)[0])*3),axis=0)

In [39]:
labels

array([1., 1., 1., ..., 3., 3., 3.])

In [40]:
np.size(labels)

4305

In [41]:
indices = np.random.permutation(data.shape[0])

# Use the shuffled indices to shuffle both data and labels
shuffled_data = data[indices]
shuffled_labels = labels[indices]
data= shuffled_data
labels=shuffled_labels


In [42]:
labels

array([4., 0., 3., ..., 2., 4., 3.])

In [43]:
from sklearn.model_selection import train_test_split

# Assume stft_data is your STFT results stored in a variable
# with shape (7530, 12, 129, 9)

# Generate labels for your data (assuming you have labels available)


# Split the data and labels into training and test sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=20)


print(f"x_train shape: {x_train.shape}")  # should be (6024, 12, 129, 9)
print(f"y_train shape: {y_train.shape}")  # should be (6024,)
print(f"x_test shape: {x_test.shape}")    # should be (1506, 12, 129, 9)
print(f"y_test shape: {y_test.shape}")    # should be (1506,)


x_train shape: (3444, 12, 2000)
y_train shape: (3444,)
x_test shape: (861, 12, 2000)
y_test shape: (861,)


In [44]:
# from keras.utils import to_categorical
# y_train1 = to_categorical(y_train)
# y_test1 = to_categorical(y_test, num_classes=5)
# print(f"x_test shape: {y_train1.shape}")    # should be (1506, 12, 129, 9)
# print(f"y_test shape: {y_test.shape}")

In [45]:
# x_train = x_train.reshape((1720,12,129*25))
# x_test = x_test.reshape((430,12,129*25))

data = dataall.reshape(dataall.shape[0], dataall.shape[1], dataall.shape[2], 1)
print(data.shape)

(4305, 12, 2000, 1)


In [47]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=2)

In [48]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import DepthwiseConv2D, Flatten, Dense


# reshape the data to have 4 dimensions

data = dataall.reshape(dataall.shape[0], dataall.shape[1], dataall.shape[2], 1)

# create labels


# # split the data into training and testing sets
# train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=2)


# create the model
model = Sequential()
model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu', input_shape=(12, 2000, 1)))
# model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu', input_shape=(12, 2000, 1)))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))
# model = Sequential()
# model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu', input_shape=(12, 2000, 1)))
# model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu'))
# model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu'))
# model.add(Flatten())
# model.add(Dense(5, activation='softmax'))


# compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(train_data, train_labels, epochs=50, batch_size=1024, validation_split=0.2)

Epoch 1/50
3/3 [==============================] - 3s 672ms/step - loss: 2.9001 - accuracy: 0.2054 - val_loss: 2.4042 - val_accuracy: 0.2177
Epoch 2/50
3/3 [==============================] - 2s 584ms/step - loss: 2.9480 - accuracy: 0.2054 - val_loss: 2.6668 - val_accuracy: 0.2104
Epoch 3/50
3/3 [==============================] - 2s 607ms/step - loss: 2.1764 - accuracy: 0.2047 - val_loss: 2.2958 - val_accuracy: 0.1713
Epoch 4/50
3/3 [==============================] - 2s 605ms/step - loss: 2.2912 - accuracy: 0.2029 - val_loss: 1.6833 - val_accuracy: 0.2104
Epoch 5/50
3/3 [==============================] - 2s 586ms/step - loss: 1.7174 - accuracy: 0.2105 - val_loss: 2.0198 - val_accuracy: 0.2177
Epoch 6/50
3/3 [==============================] - 2s 596ms/step - loss: 1.9671 - accuracy: 0.2058 - val_loss: 1.7283 - val_accuracy: 0.1785
Epoch 7/50
3/3 [==============================] - 2s 591ms/step - loss: 1.7124 - accuracy: 0.2080 - val_loss: 1.7756 - val_accuracy: 0.2104
Epoch 8/50
3/3 [====

: 

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix


# Evaluate the model on the test set with multi-class labels
test_loss_multi, test_acc_multi = model.evaluate(x_test, y_test1, verbose=2)
print('Test accuracy multi:', test_acc_multi)

# Convert y_test1 to a multi-class format
y_pred_multi = model.predict(x_test)
y_test_pred_multi = np.argmax(y_pred_multi, axis=1)

# Compute the confusion matrix for multi-class
print('Multi-class:', confusion_matrix(np.argmax(y_test1, axis=1), y_test_pred_multi))

# Evaluate the model on the test set with normal labels
test_loss_normal, test_acc_normal = model.evaluate(x_test, y_test1, verbose=2)
print('Test accuracy normal:', test_acc_normal)

# Convert y_test to a multi-class format
y_pred_normal = model.predict(x_test)
y_test_pred_normal = np.argmax(y_pred_normal, axis=1)

# Compute the confusion matrix for normal labels
print('Normal labels:', confusion_matrix(y_test, y_test_pred_normal))
